In [1]:
'''
Title:       Optimale Steuerung und Regelung:
Subttitle:   1. Aufgabe
Author:      Stefan Kaufmann
MaNr.        51867606
Date:        01.04.2023
'''

'\nTitle:       Optimale Steuerung und Regelung:\nSubttitle:   1. Aufgabe\nAuthor:      Stefan Kaufmann\nMaNr.        51867606\nDate:        01.04.2023\n'

# Optimale Steuerung und Regelung
## 1. Übung
### Stefan Kaufmann - 51867606

In [7]:
import numpy as np
import scipy as sp
from scipy.linalg import block_diag
import matplotlib.pyplot as plt

mathematisches Pendel einer oszillierenden Punktmasse mit PID Regler    
$m \ddot{y}(t) + \omega^{2}y(t) = u(t)  \hspace{2cm} y(0) = 1, \hspace{1cm} \dot{y} = 0 \\
u(t) = K_{p}y(t) + K_{D}\dot{y}(t) + K_{I} \int_{0}^{t}y(\tau) d\tau 
$  
mit dem Kostenfunktional    
$
F(k,y(k;t)) = \frac{1}{2} \int_{0}^{\inf} (x^{T}(t)Qx(t) + ru^{2})dt
$


## a) Überführung in ein satisches Problem

$ x = 
\begin{bmatrix}
y & \dot{y} & \int_{0}^{t}y(\tau) d\tau 
\end{bmatrix} ^{T}
$

$
\dot{x} = 
\begin{bmatrix}
0 & 1 & 0  \\
\frac{-\omega^{2}}{m} & 0 & 0 \\
1 & 0 & 0
\end{bmatrix} x
+
\begin{bmatrix}
0 \\ \frac{1}{m} \\ 0
\end{bmatrix} u
$     
$
y = 
\begin{bmatrix}
1 & 0 & 0
\end{bmatrix} + 0u
$

In [9]:
# Parameter
m  = 1
w  = 2
Q = block_diag(3,4,5)
R = 6

In [17]:
# State Space System
A = np.array([[0,       1, 0],
               [-w**2/m, 0, 0],
               [1,       0, 0]])

B = np.array([0,1/m,0])

C = np.array([1,0,0])
D = 0

[ 0. -3.  1.]
